<a href="https://colab.research.google.com/github/kopachsvitlana/ab_test_analysis_python_tableau/blob/main/AB_Testing_Analyzer_From_Data_to_Decisions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import norm
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# З'єднання з Google Drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# Шлях до папки
%cd /content/drive/MyDrive/MateAcademy_(files)

/content/drive/MyDrive/MateAcademy_(files)


In [ ]:
df = pd.read_csv("project.csv")
print(df.head())
print(df.info())
print(df.describe())

         date      country   device continent         channel  test  \
0  2020-11-01    Lithuania   mobile    Europe  Organic Search     2   
1  2020-11-01    Lithuania   mobile    Europe  Organic Search     1   
2  2020-11-01  El Salvador  desktop  Americas   Social Search     2   
3  2020-11-01  El Salvador  desktop  Americas   Social Search     1   
4  2020-11-01     Slovakia   mobile    Europe     Paid Search     2   

   test_group   event_name  value  
0           2  new account      1  
1           2  new account      1  
2           1  new account      1  
3           2  new account      1  
4           2  new account      1  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800996 entries, 0 to 800995
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   date        800996 non-null  object
 1   country     800996 non-null  object
 2   device      800996 non-null  object
 3   continent   800996 non-null  object
 

In [ ]:
# Замінюємо значення в колонці 'event_name'
df['event_name'] = df['event_name'].replace('new account', 'new_account')

In [ ]:
# Збільшуємо максимальну кількість символів, що можуть відображатись в одному рядку
pd.set_option('display.max_columns', None)  # Показуємо всі стовпці
pd.set_option('display.width', 200)  # Ширина екрану для відображення
pd.set_option('display.max_colwidth', None)  # Дозволяє виводити всі символи в стовпці
pd.set_option('display.max_rows', None)  # Показуємо всі рядки

In [ ]:
# Групуємо дані за test_group, test та event_name, підсумовуючи value
df_grouped = df.groupby(["test_group", "test", "event_name"])["value"].sum().unstack().reset_index()

print(df_grouped)

event_name  test_group  test  add_payment_info  add_shipping_info  add_to_cart  begin_checkout  click  first_visit  new_account  page_view  scroll  select_item  select_promotion  session_start  \
0                    1     1              1988               3034         1395            3784    368        30596         3823     191543   73244          543              1275          45905   
1                    1     2              2344               3480         2811            4262    337        34511         4165     220275   80713          905              1477          51219   
2                    1     3              3623               5298        17674            9532    280        50438         5856     286351  110360         8735              2020          71312   
3                    1     4              3731               5128        21536           12555    285        80900         8984     379480  136037        12214              2773         107128   
4                   

##Розрахунок значущості в тоталі


In [ ]:

# Функція для обчислення результатів для всіх тестів
def calculate_metrics_for_all_tests(df_grouped, metrics):
    # Ініціалізуємо список для збереження результатів
    results = []

    # Перебираємо кожен тест (test від 1 до 4)
    for test_id in range(1, 5):  # для тестів 1, 2, 3, 4
        # Фільтруємо дані для test_group 1 і 2
        df_filtered = df_grouped[(df_grouped["test"] == test_id) & (df_grouped["test_group"].isin([1, 2]))].copy()

        # Перебираємо кожну метрику
        for metric in metrics:
            # Отримуємо дані для двох груп
            x1 = df_filtered[df_filtered["test_group"] == 1][metric].values[0]  # Кількість успіхів у групі 1
            n1 = df_filtered[df_filtered["test_group"] == 1]["sessions"].values[0]  # Всього сесій у групі 1
            x2 = df_filtered[df_filtered["test_group"] == 2][metric].values[0]  # Кількість успіхів у групі 2
            n2 = df_filtered[df_filtered["test_group"] == 2]["sessions"].values[0]  # Всього сесій у групі 2

            # Розраховуємо конверсії
            p1 = x1 / n1
            p2 = x2 / n2

            # Розраховуємо pooled proportion (спільну конверсію)
            p_pooled = (x1 + x2) / (n1 + n2)

            # Розраховуємо стандартну помилку
            SE = (p_pooled * (1 - p_pooled) * (1/n1 + 1/n2)) ** 0.5

            # Розраховуємо Z-статистику
            Z = (p2 - p1) / SE

            # Розраховуємо p-value
            p_value = norm.sf(abs(Z)) * 2  # Двосторонній тест

            # Відсоткову зміну
            percentage_change = ((p2 / p1) - 1) * 100

            # Визначаємо, чи є результат статистично значущим
            significant = "True" if p_value < 0.05 else "False"

            # Зберігаємо результат
            results.append({
                "test_id": test_id,
                "metric": metric,
                "conversion_rate_1": p1,
                "conversion_rate_2": p2,
                "z_stat": Z,
                "p_value": p_value,
                "percentage_change": percentage_change,
                "significant": significant
            })

    # Перетворюємо в DataFrame для зручного виводу
    df_results = pd.DataFrame(results)
    return df_results

# Визначаємо список метрик
metrics = ["add_payment_info", "add_shipping_info", "begin_checkout", "new_account"]

# Викликаємо функцію для обчислення результатів для всіх тестів
df_results = calculate_metrics_for_all_tests(df_grouped, metrics)

# Виводимо результат
print(df_results)


    test_id             metric  conversion_rate_1  conversion_rate_2    z_stat   p_value  percentage_change significant
0         1   add_payment_info           0.043825           0.049322  3.924884  0.000087          12.542021        True
1         1  add_shipping_info           0.066884           0.071272  2.603571  0.009226           6.560481        True
2         1     begin_checkout           0.083418           0.088974  2.978783  0.002894           6.660587        True
3         1        new_account           0.084278           0.081451 -1.542883  0.122859          -3.354299       False
4         2   add_payment_info           0.046290           0.047946  1.240994  0.214608           3.576911       False
5         2  add_shipping_info           0.068724           0.069859  0.709557  0.477979           1.650995       False
6         2     begin_checkout           0.084168           0.085841  0.952898  0.340642           1.988164       False
7         2        new_account          

##Висновки

Загальні розрахунки для **тесту №1** показали, що ми отримали позитивні статистично значущі результати по трьом метрикам. Варто проаналізувати чому тест негативно повпливав на метрику new_account або для підвищення цієї метрики розробити інший тест.

Загальні розрахунки для **тесту №2** показали, що ми отримали позитивні результати, але всі вони не є статистично значущими. Тому потрібно розрахувати на скільки нам потрібно продовжити тест, щоб отримати статистично значущі результати. Якщо термін продовження нам підходить, то продовжуємо тест, а якщо не підходить - відмовляємося від даного тесту.

Загальні розрахунки для **тесту №3** показали, що ми отримали лише один позитивний результат, але він не є статистично значущим. Тому потрібно розрахувати на скільки нам потрібно продовжити тест, щоб отримати статистично значущі результати. Якщо термін продовження нам підходить, то продовжуємо тест, а якщо не підходить - відмовляємося від даного тесту.

Загальні розрахунки для **тесту №4** показали, що ми взагалі не отримали позитивних результатів, але для двох метрик вони не є статистично значущими. Тому потрібно розрахувати на скільки нам потрібно продовжити тест, щоб отримати статистично значущі результати для метрик begin_checkout та new_account. Якщо термін продовження нам підходить, то продовжуємо тест, а якщо не підходить - відмовляємося від даного тесту. Для двох інших метрик, які показали негативні статистично значущі результати пропоную провести новий тест.

##Розрахунок значущості в розрізі device

In [ ]:

# Функція для обчислення метрик по девайсах
def calculate_metrics_for_devices(df, metrics):
    # Ініціалізуємо список для збереження результатів
    results_by_device = []

    # Перебираємо кожен тест (test від 1 до 4)
    for test_id in range(1, 5):  # для тестів 1, 2, 3, 4
        # Фільтруємо дані для конкретного тесту і групуємо по тест-групам, девайсах та метриках
        df_device_grouped = df[df["test"] == test_id].groupby(["test_group", "device", "event_name"])["value"].sum().unstack().reset_index()

        for device in df_device_grouped["device"].unique():
            # Фільтруємо по девайсу
            df_device = df_device_grouped[df_device_grouped["device"] == device]

            # Перебираємо кожну метрику
            for metric in metrics:
                # Отримуємо дані для двох груп
                x1 = df_device[df_device["test_group"] == 1][metric].values[0]  # Кількість успіхів у групі 1
                n1 = df_device[df_device["test_group"] == 1]["sessions"].values[0]  # Всього сесій у групі 1
                x2 = df_device[df_device["test_group"] == 2][metric].values[0]  # Кількість успіхів у групі 2
                n2 = df_device[df_device["test_group"] == 2]["sessions"].values[0]  # Всього сесій у групі 2

                # Розраховуємо конверсії
                p1 = x1 / n1
                p2 = x2 / n2

                # Розраховуємо pooled proportion (спільну конверсію)
                p_pooled = (x1 + x2) / (n1 + n2)

                # Розраховуємо стандартну помилку
                SE = (p_pooled * (1 - p_pooled) * (1/n1 + 1/n2)) ** 0.5

                # Розраховуємо Z-статистику
                Z = (p2 - p1) / SE

                # Розраховуємо p-value
                p_value = norm.sf(abs(Z)) * 2  # Двосторонній тест

                # Відсоткову зміну
                percentage_change = ((p2 / p1) - 1) * 100

                # Визначаємо, чи є результат статистично значущим
                significant = "True" if p_value < 0.05 else "False"

                # Зберігаємо результат
                results_by_device.append({
                    "test_id": test_id,
                    "metric": metric,
                    "device": device,
                    "x1": x1,  # Кількість успіхів у групі 1
                    "n1": n1,  # Всього сесій у групі 1
                    "conversion_rate_1": p1,
                    "x2": x2,  # Кількість успіхів у групі 2
                    "n2": n2,  # Всього сесій у групі 2
                    "conversion_rate_2": p2,
                    "z_stat": Z,
                    "p_value": p_value,
                    "percentage_change": percentage_change,
                    "significant": significant
                })

    # Перетворюємо в DataFrame для зручного виводу
    df_results_by_device = pd.DataFrame(results_by_device)

    # Групуємо за пристроєм, щоб переглянути результати окремо
    device_tables = {device: df_results_by_device[df_results_by_device["device"] == device] for device in df_results_by_device["device"].unique()}

    return device_tables

# Визначаємо список метрик
metrics = ["add_payment_info", "add_shipping_info", "begin_checkout", "new_account"]

# Викликаємо функцію для обчислення результатів для всіх девайсів
device_tables = calculate_metrics_for_devices(df, metrics)

# Виводимо таблиці для кожного пристрою
for device, table in device_tables.items():
    print(f"Results for device: {device}")
    print(table, "\n")


Results for device: desktop
    test_id             metric   device      x1       n1  conversion_rate_1      x2       n2  conversion_rate_2    z_stat   p_value  percentage_change significant
0         1   add_payment_info  desktop  1130.0  26467.0           0.042695  1256.0  26417.0           0.047545  2.686998  0.007210          11.360819        True
1         1  add_shipping_info  desktop  1711.0  26467.0           0.064647  1916.0  26417.0           0.072529  3.586024  0.000336          12.193247        True
2         1     begin_checkout  desktop  2108.0  26467.0           0.079646  2404.0  26417.0           0.091002  4.673980  0.000003          14.257595        True
3         1        new_account  desktop  2203.0  26467.0           0.083236  2147.0  26417.0           0.081273 -0.821212  0.411526          -2.357527       False
12        2   add_payment_info  desktop  1314.0  29497.0           0.044547  1401.0  29380.0           0.047686  1.815587  0.069434           7.045601       

##Висновки

Розрахунки в розрізі пристроїв для **тесту №1** показали доволі негативні результати майже по всім метрикам для tablet. Пропоную впровадити зміни для всіх користувачів за виключенням device = tablet.

Розрахунки в розрізі пристроїв для **тесту №2** майже всі статистично незначущі, тому можемо розрахувати на скільки нам потрібно продовжити тест, щоб отримати статистично значущі результати. Якщо термін продовження нам підходить, то продовжуємо тест, а якщо не підходить - відмовляємося від даного тесту.

Розрахунки в розрізі пристроїв для **тесту №3** майже всі статистично незначущі, тому можемо розрахувати на скільки нам потрібно продовжити тест, щоб отримати статистично значущі результати. Якщо термін продовження нам підходить, то продовжуємо тест, а якщо не підходить - відмовляємося від даного тесту.

Розрахунки в розрізі пристроїв для **тесту №4** майже всі мають негативний статистично значущий результат, потрібно зрозуміти причини негативного ефекту  і протестувати покращену версію.

##Розрахунок значущості в розрізі channel

In [ ]:

# Функція для обчислення метрик по каналах
def calculate_metrics_for_channels(df, metrics):
    # Ініціалізуємо список для збереження результатів
    results_by_channel = []

    # Перебираємо кожен тест (test від 1 до 4)
    for test_id in range(1, 5):  # для тестів 1, 2, 3, 4
        # Фільтруємо дані для конкретного тесту і групуємо по тест-групам, каналах та метриках
        df_channel_grouped = df[df["test"] == test_id].groupby(["test_group", "channel", "event_name"])["value"].sum().unstack().reset_index()

        for channel in df_channel_grouped["channel"].unique():
            # Фільтруємо по каналу
            df_channel = df_channel_grouped[df_channel_grouped["channel"] == channel]

            # Перебираємо кожну метрику
            for metric in metrics:
                # Отримуємо дані для двох груп
                x1 = df_channel[df_channel["test_group"] == 1][metric].values[0]  # Кількість успіхів у групі 1
                n1 = df_channel[df_channel["test_group"] == 1]["sessions"].values[0]  # Всього сесій у групі 1
                x2 = df_channel[df_channel["test_group"] == 2][metric].values[0]  # Кількість успіхів у групі 2
                n2 = df_channel[df_channel["test_group"] == 2]["sessions"].values[0]  # Всього сесій у групі 2

                # Розраховуємо конверсії
                p1 = x1 / n1
                p2 = x2 / n2

                # Розраховуємо pooled proportion (спільну конверсію)
                p_pooled = (x1 + x2) / (n1 + n2)

                # Розраховуємо стандартну помилку
                SE = (p_pooled * (1 - p_pooled) * (1/n1 + 1/n2)) ** 0.5

                # Розраховуємо Z-статистику
                Z = (p2 - p1) / SE

                # Розраховуємо p-value
                p_value = norm.sf(abs(Z)) * 2  # Двосторонній тест

                # Відсоткову зміну
                percentage_change = ((p2 / p1) - 1) * 100

                # Визначаємо, чи є результат статистично значущим
                significant = "True" if p_value < 0.05 else "False"

                # Зберігаємо результат
                results_by_channel.append({
                    "test_id": test_id,
                    "metric": metric,
                    "channel": channel,
                    "x1": x1,  # Кількість успіхів у групі 1
                    "n1": n1,  # Всього сесій у групі 1
                    "conversion_rate_1": p1,
                    "x2": x2,  # Кількість успіхів у групі 2
                    "n2": n2,  # Всього сесій у групі 2
                    "conversion_rate_2": p2,
                    "z_stat": Z,
                    "p_value": p_value,
                    "percentage_change": percentage_change,
                    "significant": significant
                })

    # Перетворюємо в DataFrame для зручного виводу
    df_results_by_channel = pd.DataFrame(results_by_channel)

    # Групуємо за каналом, щоб переглянути результати окремо
    channel_tables = {channel: df_results_by_channel[df_results_by_channel["channel"] == channel] for channel in df_results_by_channel["channel"].unique()}

    return channel_tables

# Визначаємо список метрик
metrics = ["add_payment_info", "add_shipping_info", "begin_checkout", "new_account"]

# Викликаємо функцію для обчислення результатів для всіх каналів
channel_tables = calculate_metrics_for_channels(df, metrics)

# Виводимо таблиці для кожного каналу
for channel, table in channel_tables.items():
    print(f"Results for channel: {channel}")
    print(table, "\n")


Results for channel: Direct
    test_id             metric channel      x1       n1  conversion_rate_1      x2       n2  conversion_rate_2    z_stat   p_value  percentage_change significant
0         1   add_payment_info  Direct   392.0  10691.0           0.036666   516.0  10361.0           0.049802  4.690261  0.000003          35.825180        True
1         1  add_shipping_info  Direct   664.0  10691.0           0.062108   716.0  10361.0           0.069105  2.050707  0.040295          11.265775        True
2         1     begin_checkout  Direct   823.0  10691.0           0.076981   915.0  10361.0           0.088312  2.986589  0.002821          14.719677        True
3         1        new_account  Direct   913.0  10691.0           0.085399   850.0  10361.0           0.082038 -0.879999  0.378860          -3.935085       False
20        2   add_payment_info  Direct   516.0  11820.0           0.043655   504.0  11670.0           0.043188 -0.175651  0.860568          -1.070126       False


##Висновки

Розрахунки в розрізі каналів для **тесту №1** показали змішані резутьтати. Пропоную погоджувати тес для каналів: Direct та Undefined. А для інших каналів розглянути оптимізацію.

Розрахунки в розрізі каналів для **тесту №2** показали змішані резутьтати. Пропоную погоджувати тес для каналів: Paid Search, Social Search та Undefined. А для інших каналів розглянути оптимізацію.

Розрахунки в розрізі каналів для **тесту №3** показали позитивно статистично значущі результати лише для каналу Undefined. Тому цей тест пропоную запустити лише для одного каналу, а для інших каналів розглянути оптимізацію.

Розрахунки в розрізі каналів для **тесту №4** показали негативні або статистично незначущі результати. Пропоную відмовитися від даного тесту або доопрацювати його.

##Розрахунок значущості в розрізі continent

In [ ]:

# Функція для обчислення метрик по континентах
def calculate_metrics_for_continents(df, metrics):
    # Ініціалізуємо список для збереження результатів
    results_by_continent = []

    # Перебираємо кожен тест (test від 1 до 4)
    for test_id in range(1, 5):  # для тестів 1, 2, 3, 4
        # Фільтруємо дані для конкретного тесту і групуємо по тест-групам, континентах та метриках
        df_continent_grouped = df[df["test"] == test_id].groupby(["test_group", "continent", "event_name"])["value"].sum().unstack().reset_index()

        for continent in df_continent_grouped["continent"].unique():
            # Фільтруємо по континенту
            df_continent = df_continent_grouped[df_continent_grouped["continent"] == continent]

            # Перебираємо кожну метрику
            for metric in metrics:
                # Отримуємо дані для двох груп
                x1 = df_continent[df_continent["test_group"] == 1][metric].values[0]  # Кількість успіхів у групі 1
                n1 = df_continent[df_continent["test_group"] == 1]["sessions"].values[0]  # Всього сесій у групі 1
                x2 = df_continent[df_continent["test_group"] == 2][metric].values[0]  # Кількість успіхів у групі 2
                n2 = df_continent[df_continent["test_group"] == 2]["sessions"].values[0]  # Всього сесій у групі 2

                # Розраховуємо конверсії
                p1 = x1 / n1
                p2 = x2 / n2

                # Розраховуємо pooled proportion (спільну конверсію)
                p_pooled = (x1 + x2) / (n1 + n2)

                # Розраховуємо стандартну помилку
                SE = (p_pooled * (1 - p_pooled) * (1/n1 + 1/n2)) ** 0.5

                # Розраховуємо Z-статистику
                Z = (p2 - p1) / SE

                # Розраховуємо p-value
                p_value = norm.sf(abs(Z)) * 2  # Двосторонній тест

                # Відсоткову зміну
                percentage_change = ((p2 / p1) - 1) * 100

                # Визначаємо, чи є результат статистично значущим
                significant = "True" if p_value < 0.05 else "False"

                # Зберігаємо результат
                results_by_continent.append({
                    "test_id": test_id,
                    "metric": metric,
                    "continent": continent,
                    "x1": x1,  # Кількість успіхів у групі 1
                    "n1": n1,  # Всього сесій у групі 1
                    "conversion_rate_1": p1,
                    "x2": x2,  # Кількість успіхів у групі 2
                    "n2": n2,  # Всього сесій у групі 2
                    "conversion_rate_2": p2,
                    "z_stat": Z,
                    "p_value": p_value,
                    "percentage_change": percentage_change,
                    "significant": significant
                })

    # Перетворюємо в DataFrame для зручного виводу
    df_results_by_continent = pd.DataFrame(results_by_continent)

    # Групуємо за континентом, щоб переглянути результати окремо
    continent_tables = {continent: df_results_by_continent[df_results_by_continent["continent"] == continent] for continent in df_results_by_continent["continent"].unique()}

    return continent_tables

# Визначаємо список метрик
metrics = ["add_payment_info", "add_shipping_info", "begin_checkout", "new_account"]

# Викликаємо функцію для обчислення результатів для всіх континентів
continent_tables = calculate_metrics_for_continents(df, metrics)

# Виводимо таблиці для кожного континенту
for continent, table in continent_tables.items():
    print(f"Results for continent: {continent}")
    print(table, "\n")


Results for continent: (not set)
    test_id             metric  continent    x1     n1  conversion_rate_1    x2     n2  conversion_rate_2    z_stat   p_value  percentage_change significant
0         1   add_payment_info  (not set)   7.0   97.0           0.072165  10.0  100.0           0.100000  0.695585  0.486689          38.571429       False
1         1  add_shipping_info  (not set)   4.0   97.0           0.041237  13.0  100.0           0.130000  2.218144  0.026545         215.250000        True
2         1     begin_checkout  (not set)   4.0   97.0           0.041237  22.0  100.0           0.220000  3.706053  0.000211         433.500000        True
3         1        new_account  (not set)   7.0   97.0           0.072165  11.0  100.0           0.110000  0.921405  0.356839          52.428571       False
24        2   add_payment_info  (not set)  11.0   89.0           0.123596   7.0  130.0           0.053846 -1.845842  0.064915         -56.433566       False
25        2  add_shipping

##Висновки

Розрахунки в розрізі континентів для **тесту №1** показали змішані резутьтати. Пропоную погоджувати тест для континентів з групи not set та Europe. А для інших потрібно розрахувати термін продовження тесту, щоб отримати статистично значущі результати. Якщо термін продовження нам підходить, то продовжуємо тест, а якщо не підходить - відмовляємося від даного тесту на цих континентах.

Розрахунки в розрізі континентів для **тесту №2** показали доволі негативні результати. Пропоную провети оптимізацію тесту.

Розрахунки в розрізі континентів для **тесту №3** показали доволі негативні результати. Пропоную провети оптимізацію тесту.

Розрахунки в розрізі континентів для **тесту №4** показали доволі негативні результати. Пропоную провети оптимізацію тесту.

###Посилання на дашборд: https://public.tableau.com/app/profile/svitlana.kopach/viz/SignificanceresultsofABtests/Dashboard1

###Посилання на файл з результатами розрахунків першого етапу завдання:
https://docs.google.com/spreadsheets/d/1-4dwGLunngRlKR7a2ndhQVSaQdqkEqdA/edit?usp=sharing&ouid=112754362411357494984&rtpof=true&sd=true